# Module 2: Train and Evaluate a Keras-Based Classifier

In [ ]:
import os, glob, numpy as np, matplotlib.pyplot as plt
from PIL import Image, ImageDraw

DATASET_DIR = "./images_dataSAT"
DIR_NON_AGRI = os.path.join(DATASET_DIR, "class_0_non_agri")
DIR_AGRI = os.path.join(DATASET_DIR, "class_1_agri")

def _ensure_dataset():
    os.makedirs(DIR_NON_AGRI, exist_ok=True)
    os.makedirs(DIR_AGRI, exist_ok=True)
    if len(os.listdir(DIR_NON_AGRI))>0 and len(os.listdir(DIR_AGRI))>0:
        return
    import numpy as np
    from PIL import Image, ImageDraw
    rng = np.random.default_rng(0)
    for cls_dir, pattern in [(DIR_NON_AGRI, 'rect'), (DIR_AGRI, 'lines')]:
        for i in range(12):
            img = Image.new("RGB",(64,64),(rng.integers(20,235),rng.integers(20,235),rng.integers(20,235)))
            d = ImageDraw.Draw(img)
            if pattern=='rect':
                d.rectangle([10,10,54,54], outline=(255,255,255), width=2)
            else:
                for y in range(5,64,10):
                    d.line([0,y,64,y], fill=(255,255,255), width=1)
            img.save(os.path.join(cls_dir, f"img_{{i:03d}}.png"))

# Copy dataset from /mnt/data if available
if os.path.exists('/mnt/data/images_dataSAT'):
    import shutil
    if not os.path.exists(DATASET_DIR):
        shutil.copytree('/mnt/data/images_dataSAT', DATASET_DIR)
_ensure_dataset()
print("Dataset ready at", os.path.abspath(DATASET_DIR))

In [ ]:
import os, matplotlib.pyplot as plt, tensorflow as tf
from tensorflow.keras import layers, models

dataset_path = DATASET_DIR
fnames = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.lower().endswith(('.png','.jpg','.jpeg')):
            fnames.append(os.path.join(root, f))
fnames = sorted(fnames)
print("Total files found:", len(fnames)); print("First 5:", fnames[:5])

val_ds = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=(64,64), batch_size=16, validation_split=0.3, subset='validation', seed=123)
train_ds = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=(64,64), batch_size=16, validation_split=0.3, subset='training', seed=123)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE); train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(64,64,3)),
    layers.Conv2D(16, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'), layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
print("Total layers:", len(test_model.layers))
ckpt = tf.keras.callbacks.ModelCheckpoint("best.keras", monitor="val_accuracy", save_best_only=True, mode="max")
test_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = test_model.fit(train_ds, validation_data=val_ds, epochs=3, callbacks=[ckpt])
plt.figure(); plt.plot(hist.history['loss'], label='train_loss'); plt.plot(hist.history['val_loss'], label='val_loss'); plt.legend(); plt.title('Loss'); plt.show()